In [1]:
from torchvision import utils
from basic_fcn import *
from unet import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
from matplotlib import pyplot as plt
import os
from pprint import pprint

In [2]:
train_dataset = CityScapesDataset(csv_file='./Data/train.csv')
val_dataset = CityScapesDataset(csv_file='./Data/val.csv')
test_dataset = CityScapesDataset(csv_file='./Data/test.csv')

_batch_size = 4
_num_workers = 4
iou_classes = [11, 20, 24, 26, 33]

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=1,
                          num_workers=_num_workers,
                          shuffle=True)

In [3]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform(m.bias.data)
        m.bias.data.zero_()
        
epochs     = 200
criterion = nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
unet_model = Unet(n_class=n_class)
    
unet_model.apply(init_weights)
#fcn_model = torch.load('best_model')

optimizer = optim.Adam(unet_model.parameters(), lr=5e-3)

In [4]:
modelName = 'unet_test'
MYDIR = "./Results" + "/" + modelName
CHECK_FOLDER = os.path.isdir(MYDIR)
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")

def plotTwoGraphs(base, train_metrics, val_metrics):
    plt.figure()
    plt.plot(base, train_metrics)
    plt.plot(base, val_metrics)
    plt.gca().legend(('train','validation'))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss vs Epochs')
    plt.savefig(MYDIR + '/loss.png')

def plotGraph(base, metrics, name):
    plt.figure()
    plt.plot(base, metrics)
    plt.xlabel('Epochs')
    plt.ylabel(name)
    plt.title(name + ' vs Epochs')
    plt.savefig(MYDIR + '/' + name + '.png')
    

./Results/unet_test folder already exists.


In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    unet_model = unet_model.cuda()
    
all_train_losses = []
all_val_losses = []
all_val_accuracies = []
all_val_iou = []
all_val_iou_class = {}


        
def train():
    startingValLoss = 10000000.0
    for epoch in range(epochs):
        unet_model.train()
        curEpochLoss = []
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()   # Move your inputs onto the gpu
                labels = Y.cuda() # Move your labels onto the gpu
            else:
                inputs, labels = X, Y# Unpack variables into inputs and labels
            outputs = unet_model(inputs)
            loss = criterion(outputs, labels)
            curEpochLoss.append(loss.item())
            loss.backward()
            optimizer.step()

            if iter % 100 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item(), time.time() - ts))
        curEpochLoss = sum(curEpochLoss)/len(curEpochLoss)
        print('training loss : ', curEpochLoss)
        all_train_losses.append(curEpochLoss)
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        
        
        val_loss, val_accuracy, val_iou_net, val_iou_net_class_wise = val(epoch)
        all_val_losses.append(val_loss)
        all_val_accuracies.append(val_accuracy)
        all_val_iou.append(val_iou_net)
        for c in val_iou_net_class_wise:
            if(c not in all_val_iou_class):
                all_val_iou_class[c] = []
            all_val_iou_class[c].append(val_iou_net_class_wise[c])
        
        plotTwoGraphs([i for i in range(epoch+1)], all_train_losses, all_val_losses)
        plotGraph([i for i in range(epoch+1)], all_val_accuracies, 'validation_accuracy')
        plotGraph([i for i in range(epoch+1)], all_val_iou, 'average_iou')
        for c in all_val_iou_class:
            plotGraph([i for i in range(epoch+1)], all_val_iou_class[c], 'iou'+str(c))
        
        if(val_loss < startingValLoss):
            startingValLoss = val_loss
            results = {}
            results['accuracy'] = val_accuracy
            results['iou_net'] = val_iou_net
            results['iou_net_class_wise'] = val_iou_net_class_wise
            results['total_loss'] = val_loss
            results['epoch'] = epoch
            with open(MYDIR + '/results_val.txt', 'wt') as out:
                pprint(results, stream=out)
            torch.save(unet_model, MYDIR + '/best_model')
            #test(epoch)
    


def val(epoch):
    unet_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    total_loss = []
    acc_num = 0
    acc_den = 0
    #iou_num = len(iou_classes)*[0] 
    #iou_den = len(iou_classes)*[0] 
    iou_num = {}
    iou_den = {}
    
    for iter, (X_val, tar_val, Y_val) in enumerate(val_loader):
        if use_gpu:
            inputs_val = X_val.cuda()   # Move your inputs onto the gpu
            labels_val = Y_val.cuda() # Move your labels onto the gpu
        else:
            inputs_val, labels_val = X_val, Y_val# Unpack variables into inputs and labels

        outputs_val = unet_model(inputs_val)
        
        _, preds = torch.max(outputs_val, 1)
        
        loss_val = criterion(outputs_val, labels_val).item()
        
        preds_cpu = preds.cpu()
        Y_cpu = Y_val.cpu()
        
        total_loss.append(loss_val)
        
        nn, dd = pixel_acc(preds_cpu, Y_cpu)
        acc_num += nn
        acc_den += dd
        
        for iou_cc_idx in range(len(iou_classes)):
            i_nn, i_dd = iou(preds_cpu, Y_cpu, iou_classes[iou_cc_idx])
            if iou_cc_idx not in iou_num:
                iou_num[iou_cc_idx] = 0
                iou_den[iou_cc_idx] = 0
            iou_num[iou_cc_idx] += i_nn
            iou_den[iou_cc_idx] += i_dd
            
        del inputs_val
        del labels_val
        del loss_val
    total_loss = sum(total_loss)/len(total_loss)
    print("Validation loss after epoch {}, , loss: {}".format(epoch, total_loss))
    print("Accuracy on val after epoch {}, accuracy: {}".format(epoch, 100*acc_num/acc_den))
    for iou_cc_idx in range(len(iou_classes)):
        print("IOU for class {} on val after epoch{}, iou: {}".format(iou_classes[iou_cc_idx], epoch, 100*iou_num[iou_cc_idx]/iou_den[iou_cc_idx]))

    iou_net_class_wise = dict((c, iou_num[c]/iou_den[c]) for c in iou_num)
    accuracy = acc_num / acc_den
    iou_net_num = 0
    iou_net_den = 0
    for c in iou_num:
        iou_net_num += iou_num[c]
        iou_net_den += iou_den[c]
    iou_net = iou_net_num/iou_net_den

    return total_loss, accuracy, iou_net, iou_net_class_wise

def colorMap(a, i):
    return idsColor[a][i]

def test(numFigures):
    #fcn_model.load_state_dict(torch.load(MYDIR + '/best_model'))
    model = torch.load(MYDIR + '/best_model')
    model.cuda()
    model.eval()
    dataiter = iter(test_loader)
    i = 0
    while(i < numFigures):
        i += 1
        X_test, img_full, Y_test = dataiter.next()
        if use_gpu:
            inputs_test = X_test.cuda()   # Move your inputs onto the gpu
            labels_test = Y_test.cuda() # Move your labels onto the gpu
            #img_full_test = img_full.cuda()
        else:
            inputs_test, labels_test = X_test, Y_test# Unpack variables into inputs and labels
            #img_full_test = img_full

        outputs = model(inputs_test)
        _, preds = torch.max(outputs, 1)
        inputs_test = inputs_test.permute(0, 2, 3, 1)
        #img_full_test = img_full_test.permute(0, 2, 3, 1)

        f = plt.figure(figsize=(16,8))
        ax = f.add_subplot(121)
        ax2 = f.add_subplot(122)

        #plt.subplot(1, 2, 1)
        ax.imshow(torchvision.utils.make_grid(-1*inputs_test.cpu()))


        preds = preds.cpu().numpy()

        preds0 = np.vectorize(colorMap)(preds, 0)
        preds1 = np.vectorize(colorMap)(preds, 1)
        preds2 = np.vectorize(colorMap)(preds, 2)

        preds = torch.from_numpy(np.transpose(np.concatenate((preds0, preds1, preds2), 0), (1,2,0)))

        #plt.subplot(1, 2, 2)
        ax2.imshow(torchvision.utils.make_grid(preds.cpu()))
        
    
if __name__ == "__main__":
    #test(10)
    train()

epoch0, iter0, loss: 4.081492900848389
epoch0, iter100, loss: 2.0944466590881348
epoch0, iter200, loss: 2.2188498973846436
epoch0, iter300, loss: 1.779818058013916
epoch0, iter400, loss: 1.595750331878662
epoch0, iter500, loss: 1.199023962020874
epoch0, iter600, loss: 2.11834979057312
epoch0, iter700, loss: 1.6761348247528076
training loss :  1.8260249934529746
Finish epoch 0, time elapsed 169.2627830505371
Validation loss after epoch 0, , loss: 1.4780387501716614
Accuracy on val after epoch 0, accuracy: 63.59333202060975
IOU for class 11 on val after epoch0, iou: 33.545494565236304
IOU for class 20 on val after epoch0, iou: 0.0
IOU for class 24 on val after epoch0, iou: 0.0
IOU for class 26 on val after epoch0, iou: 9.050692632281564
IOU for class 33 on val after epoch0, iou: 0.0


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Unet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type MaxPool2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.7/s

epoch1, iter0, loss: 2.38748836517334
epoch1, iter100, loss: 1.7992254495620728
epoch1, iter200, loss: 1.3053669929504395
epoch1, iter300, loss: 1.4601860046386719
epoch1, iter400, loss: 2.062434434890747
epoch1, iter500, loss: 0.8591049909591675
epoch1, iter600, loss: 1.4901260137557983
epoch1, iter700, loss: 1.8906562328338623
training loss :  1.5729828686002763
Finish epoch 1, time elapsed 170.83172488212585
Validation loss after epoch 1, , loss: 1.5352466065883636
Accuracy on val after epoch 1, accuracy: 64.25957656037188
IOU for class 11 on val after epoch1, iou: 31.821126464383347
IOU for class 20 on val after epoch1, iou: 2.7644518914738674
IOU for class 24 on val after epoch1, iou: 0.0
IOU for class 26 on val after epoch1, iou: 16.468974359560487
IOU for class 33 on val after epoch1, iou: 0.0
epoch2, iter0, loss: 1.73259437084198
epoch2, iter100, loss: 1.53183913230896
epoch2, iter200, loss: 1.6283568143844604
epoch2, iter300, loss: 1.3882298469543457
epoch2, iter400, loss: 0.9

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


epoch3, iter0, loss: 1.1240882873535156
epoch3, iter100, loss: 1.8242173194885254
epoch3, iter200, loss: 1.055253267288208
epoch3, iter300, loss: 1.2855017185211182
epoch3, iter400, loss: 0.9017537236213684
epoch3, iter500, loss: 0.9804880619049072
epoch3, iter600, loss: 1.4695030450820923
epoch3, iter700, loss: 1.260101079940796
training loss :  1.465376484658449
Finish epoch 3, time elapsed 171.168518781662
Validation loss after epoch 3, , loss: 1.3309197087287903
Accuracy on val after epoch 3, accuracy: 69.45245656361679
IOU for class 11 on val after epoch3, iou: 42.72280361578731
IOU for class 20 on val after epoch3, iou: 5.198948228898916
IOU for class 24 on val after epoch3, iou: 1.514592878499966
IOU for class 26 on val after epoch3, iou: 18.802462874077662
IOU for class 33 on val after epoch3, iou: 0.0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


epoch4, iter0, loss: 1.3482089042663574
epoch4, iter100, loss: 1.6923010349273682
epoch4, iter200, loss: 1.5227125883102417
epoch4, iter300, loss: 1.7618281841278076
epoch4, iter400, loss: 0.66685551404953
epoch4, iter500, loss: 0.9891368746757507
epoch4, iter600, loss: 1.3625450134277344
epoch4, iter700, loss: 1.874997615814209
training loss :  1.4037151485681534
Finish epoch 4, time elapsed 171.40306568145752
Validation loss after epoch 4, , loss: 1.2890182123184204
Accuracy on val after epoch 4, accuracy: 71.12674156029578
IOU for class 11 on val after epoch4, iou: 46.341004323687905
IOU for class 20 on val after epoch4, iou: 3.9969166262129447
IOU for class 24 on val after epoch4, iou: 3.661523704460474
IOU for class 26 on val after epoch4, iou: 23.23316462386716
IOU for class 33 on val after epoch4, iou: 0.0
epoch5, iter0, loss: 1.032326340675354
epoch5, iter100, loss: 0.8852105736732483
epoch5, iter200, loss: 1.6554343700408936
epoch5, iter300, loss: 1.5482892990112305
epoch5, it

epoch14, iter0, loss: 0.9511656165122986
epoch14, iter100, loss: 1.1408525705337524
epoch14, iter200, loss: 1.1535533666610718
epoch14, iter300, loss: 1.5981515645980835
epoch14, iter400, loss: 0.5084760189056396
epoch14, iter500, loss: 0.6437076330184937
epoch14, iter600, loss: 0.6471109390258789
epoch14, iter700, loss: 2.2500033378601074
training loss :  1.1436927441147067
Finish epoch 14, time elapsed 171.0954623222351
Validation loss after epoch 14, , loss: 1.178815882921219
Accuracy on val after epoch 14, accuracy: 71.74905865682413
IOU for class 11 on val after epoch14, iou: 47.463610743747694
IOU for class 20 on val after epoch14, iou: 7.652082256686298
IOU for class 24 on val after epoch14, iou: 0.9676417400782421
IOU for class 26 on val after epoch14, iou: 20.677427509262444
IOU for class 33 on val after epoch14, iou: 0.0
epoch15, iter0, loss: 0.8663854598999023
epoch15, iter100, loss: 0.68011873960495
epoch15, iter200, loss: 0.7326658368110657
epoch15, iter300, loss: 1.608652

epoch24, iter0, loss: 0.6169191598892212
epoch24, iter100, loss: 1.034409999847412
epoch24, iter200, loss: 1.7847416400909424
epoch24, iter300, loss: 0.8886474370956421
epoch24, iter400, loss: 0.6118707060813904
epoch24, iter500, loss: 0.3559900224208832
epoch24, iter600, loss: 0.9160225987434387
epoch24, iter700, loss: 1.818580985069275
training loss :  0.9679832958806587
Finish epoch 24, time elapsed 171.30221223831177
Validation loss after epoch 24, , loss: 0.9377117544412613
Accuracy on val after epoch 24, accuracy: 80.9256008320052
IOU for class 11 on val after epoch24, iou: 55.78980725105738
IOU for class 20 on val after epoch24, iou: 5.909428695528662
IOU for class 24 on val after epoch24, iou: 5.449211075304417
IOU for class 26 on val after epoch24, iou: 46.13035155736947
IOU for class 33 on val after epoch24, iou: 0.0
epoch25, iter0, loss: 1.0111998319625854
epoch25, iter100, loss: 1.190350890159607
epoch25, iter200, loss: 0.3912842571735382
epoch25, iter300, loss: 1.231588125